id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

Loading the dataset

In [2]:
import pickle
dataset = pickle.load(open("datasets/n 300 - seed 1234 - time 2019-06-19-153241.pickle","rb"))

Describing the data

In [2]:
print("Number of datasets generated is %d \n" % dataset['n'])

print("Coefficients")
print(dataset['c'])

print("\n Data preview")
dataset['data'][0].head()


NameError: name 'dataset' is not defined

Running the model

In [3]:
#!pip install pathvalidate
#!pip install joblib
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")
mixedlogit.fit(drawtype="pseudo", n_draws=5, verbose=2, save="sagetest/p", end=5, parallel=True)
# mixedlogit = Mixedlogit(dataset, "bigbatch").fit(drawtype="pseudo", n_draws=2, verbose=1, save="test 2", start=3, end=6)


Fitting 5 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   35.7s remaining:   53.6s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   41.7s remaining:   27.8s


Finished parallel excuting of 5 jobs in 0:00:55, time per job 0:00:11

Initializing save
Saving run 1 of 5 

Saving run 2 of 5 

Saving run 3 of 5 

Saving run 4 of 5 

Saving run 5 of 5 

All results saved to pickle: Results/sagetest/p - dgp bigbatch- start 0 end 4 - drawtype pseudo - ndraws 5- timestamp 2019-06-20-145653


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   55.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   55.2s finished


In [ ]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")

# #ndraws 50
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=2, save="Base/parallel-1", start=0, end=200, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=2, save="Base/parallel-2", start=200, end=400, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=2, save="Base/parallel-3", start=400, end=600, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=2, save="Base/parallel-4", start=600, end=800, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=2, save="Base/parallel-5", start=800, end=1000, parallel=True)

# #ndraws 100
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=2, save="Base/parallel-1", start=0, end=200, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=2, save="Base/parallel-2", start=200, end=400, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=2, save="Base/parallel-3", start=400, end=600, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=2, save="Base/parallel-4", start=600, end=800, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=2, save="Base/parallel-5", start=800, end=1000, parallel=True)

Fitting 200 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  78 out of 200 | elapsed:  9.8min remaining: 15.3min
[Parallel(n_jobs=-1)]: Done  99 out of 200 | elapsed: 11.6min remaining: 11.8min
[Parallel(n_jobs=-1)]: Done 120 out of 200 | elapsed: 12.8min remaining:  8.5min
[Parallel(n_jobs=-1)]: Done 141 out of 200 | elapsed: 14.5min remaining:  6.1min
[Parallel(n_jobs=-1)]: Done 162 out of 200 | elapsed: 16.5min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done 183 out of 200 | elapsed: 17.9min remaining:  1.7min


Finished parallel excuting of 200 jobs in 0:22:40, time per job 0:00:07

Initializing save
Saving run 1 of 200 

Saving run 2 of 200 

Saving run 3 of 200 

Saving run 4 of 200 

Saving run 5 of 200 

Saving run 6 of 200 

Saving run 7 of 200 

Saving run 8 of 200 

Saving run 9 of 200 

Saving run 10 of 200 

Saving run 11 of 200 

Saving run 12 of 200 

Saving run 13 of 200 

Saving run 14 of 200 

Saving run 15 of 200 

Saving run 16 of 200 

Saving run 17 of 200 

Saving run 18 of 200 

Saving run 19 of 200 

Saving run 20 of 200 

Saving run 21 of 200 

Saving run 22 of 200 

Saving run 23 of 200 

Saving run 24 of 200 

Saving run 25 of 200 

Saving run 26 of 200 

Saving run 27 of 200 

Saving run 28 of 200 

Saving run 29 of 200 

Saving run 30 of 200 

Saving run 31 of 200 

Saving run 32 of 200 

Saving run 33 of 200 

Saving run 34 of 200 

Saving run 35 of 200 

Saving run 36 of 200 

Saving run 37 of 200 

Saving run 38 of 200 

Saving run 39 of 200 

Saving run 40 of 200 

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 22.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done  78 out of 200 | elapsed:  9.5min remaining: 14.8min
[Parallel(n_jobs=-1)]: Done  99 out of 200 | elapsed: 11.5min remaining: 11.7min
[Parallel(n_jobs=-1)]: Done 120 out of 200 | elapsed: 13.7min remaining:  9.1min
[Parallel(n_jobs=-1)]: Done 141 out of 200 | elapsed: 15.1min remaining:  6.3min
[Parallel(n_jobs=-1)]: Done 162 out of 200 | elapsed: 16.5min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done 183 out of 200 | elapsed: 18.0min remaining:  1.7min


Finished parallel excuting of 200 jobs in 0:25:05, time per job 0:00:08

Initializing save
Saving run 1 of 200 

Saving run 2 of 200 

Saving run 3 of 200 

Saving run 4 of 200 

Saving run 5 of 200 

Saving run 6 of 200 

Saving run 7 of 200 

Saving run 8 of 200 

Saving run 9 of 200 

Saving run 10 of 200 

Saving run 11 of 200 

Saving run 12 of 200 

Saving run 13 of 200 

Saving run 14 of 200 

Saving run 15 of 200 

Saving run 16 of 200 

Saving run 17 of 200 

Saving run 18 of 200 

Saving run 19 of 200 

Saving run 20 of 200 

Saving run 21 of 200 

Saving run 22 of 200 

Saving run 23 of 200 

Saving run 24 of 200 

Saving run 25 of 200 

Saving run 26 of 200 

Saving run 27 of 200 

Saving run 28 of 200 

Saving run 29 of 200 

Saving run 30 of 200 

Saving run 31 of 200 

Saving run 32 of 200 

Saving run 33 of 200 

Saving run 34 of 200 

Saving run 35 of 200 

Saving run 36 of 200 

Saving run 37 of 200 

Saving run 38 of 200 

Saving run 39 of 200 

Saving run 40 of 200 

...

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 43.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done  78 out of 200 | elapsed: 17.5min remaining: 27.3min
[Parallel(n_jobs=-1)]: Done  99 out of 200 | elapsed: 21.7min remaining: 22.2min
[Parallel(n_jobs=-1)]: Done 120 out of 200 | elapsed: 25.6min remaining: 17.1min
[Parallel(n_jobs=-1)]: Done 141 out of 200 | elapsed: 29.2min remaining: 12.2min
[Parallel(n_jobs=-1)]: Done 162 out of 200 | elapsed: 31.9min remaining:  7.5min
[Parallel(n_jobs=-1)]: Done 183 out of 200 | elapsed: 35.9min remaining:  3.3min


Finished parallel excuting of 200 jobs in 0:44:29, time per job 0:00:13

Initializing save
Saving run 1 of 200 

Saving run 2 of 200 

Saving run 3 of 200 

Saving run 4 of 200 

Saving run 5 of 200 

Saving run 6 of 200 

Saving run 7 of 200 

Saving run 8 of 200 

Saving run 9 of 200 

Saving run 10 of 200 

Saving run 11 of 200 

Saving run 12 of 200 

Saving run 13 of 200 

Saving run 14 of 200 

Saving run 15 of 200 

Saving run 16 of 200 

Saving run 17 of 200 

Saving run 18 of 200 

Saving run 19 of 200 

Saving run 20 of 200 

Saving run 21 of 200 

Saving run 22 of 200 

Saving run 23 of 200 

Saving run 24 of 200 

Saving run 25 of 200 

Saving run 26 of 200 

Saving run 27 of 200 

Saving run 28 of 200 

Saving run 29 of 200 

Saving run 30 of 200 

Saving run 31 of 200 

Saving run 32 of 200 

Saving run 33 of 200 

Saving run 34 of 200 

Saving run 35 of 200 

Saving run 36 of 200 

Saving run 37 of 200 

Saving run 38 of 200 

Saving run 39 of 200 

Saving run 40 of 200 

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 44.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  78 out of 200 | elapsed: 19.3min remaining: 30.2min
[Parallel(n_jobs=-1)]: Done  99 out of 200 | elapsed: 22.7min remaining: 23.2min
[Parallel(n_jobs=-1)]: Done 120 out of 200 | elapsed: 25.4min remaining: 16.9min
[Parallel(n_jobs=-1)]: Done 141 out of 200 | elapsed: 29.4min remaining: 12.3min
[Parallel(n_jobs=-1)]: Done 162 out of 200 | elapsed: 32.8min remaining:  7.7min
[Parallel(n_jobs=-1)]: Done 183 out of 200 | elapsed: 36.3min remaining:  3.4min


Finished parallel excuting of 200 jobs in 1:07:13, time per job 0:00:20

Initializing save
Saving run 1 of 200 

Saving run 2 of 200 

Saving run 3 of 200 

Saving run 4 of 200 

Saving run 5 of 200 

Saving run 6 of 200 

Saving run 7 of 200 

Saving run 8 of 200 

Saving run 9 of 200 

Saving run 10 of 200 

Saving run 11 of 200 

Saving run 12 of 200 

Saving run 13 of 200 

Saving run 14 of 200 

Saving run 15 of 200 

Saving run 16 of 200 

Saving run 17 of 200 

Saving run 18 of 200 

Saving run 19 of 200 

Saving run 20 of 200 

Saving run 21 of 200 

Saving run 22 of 200 

Saving run 23 of 200 

Saving run 24 of 200 

Saving run 25 of 200 

Saving run 26 of 200 

Saving run 27 of 200 

Saving run 28 of 200 

Saving run 29 of 200 

Saving run 30 of 200 

Saving run 31 of 200 

Saving run 32 of 200 

Saving run 33 of 200 

Saving run 34 of 200 

Saving run 35 of 200 

Saving run 36 of 200 

Saving run 37 of 200 

Saving run 38 of 200 

Saving run 39 of 200 

Saving run 40 of 200 

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 67.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
